# Species Analysis

## import

In [1]:
import os

In [2]:
import pandas as pd

In [3]:
import matplotlib.pyplot as plt

In [4]:
import numpy as np

In [5]:
import json

In [6]:
%run ../../Util/FileFolder

created


In [7]:
%run ../../Util/Variable

# subfolders

In [8]:
subfolder={}

## sub folder count

In [9]:
subfolderCount=num_sub_folders(original_dataset_dir)
subfolder['count']=subfolderCount

In [10]:
today_collection_path=os.path.join(today_summary_path,'collection')


In [11]:
overviewTxt=os.path.join(today_collection_path,'Overview.txt')

In [12]:
with open (overviewTxt,'a') as overviewFile:
    overviewFile.write(f'species:{subfolderCount}\n')

### Subfolder names


In [13]:
folders_names=sub_folders(original_dataset_dir)

In [14]:
subfoldersNamesTxt=os.path.join(today_collection_path,'subfolders_Names.txt')

In [15]:
with open (subfoldersNamesTxt,'a') as subfoldersFile:
    for subfolder in folders_names:
        subfoldersFile.write(f'{subfolder}\n')

In [16]:
folders=list(
    map(
        lambda name: 
            {'name':name,
             'path':os.path.join(original_dataset_dir,name),
             'filecount':count_files_in_folder(os.path.join(original_dataset_dir,name)),
             'volume':{
                 'GB':folder_size_in_gb(os.path.join(original_dataset_dir,name)),
                 "KB":folder_size_in_kb(os.path.join(original_dataset_dir,name)),
                "MB":folder_size_in_mb(os.path.join(original_dataset_dir,name))
             }
            },folders_names))
folder_li=list(map( lambda name: 
            [name,
             folder_size_in_gb(os.path.join(original_dataset_dir,name)),
            folder_size_in_kb(os.path.join(original_dataset_dir,name)),
            folder_size_in_mb(os.path.join(original_dataset_dir,name))
             
            ],folders_names))

In [18]:
save_to_csv_filedetails(data=folder_li,colums=['Names','VolumeKB','VolumeMB',"VolumeGB"],csv_filename='volumereportcsv.csv',save_folder=today_collection_path)

32

### subfolder details

In [19]:
subfolders=[]

In [31]:
reportjsonfile=os.path.join(today_collection_path,'reportjsonfile.json')

In [32]:
with open(reportjsonfile, 'r+') as f:
        # Load existing data from JSON
        existing_data = json.load(f)
        
        # Append new data to existing JSON data
        existing_data['subfolder']=subfolders
        
        # Write updated data back to JSON file
        f.seek(0)
        json.dump(existing_data, f, indent=4)
        f.truncate()

In [29]:
subfolderdetails =list(map(lambda folder: {
    'name':folder['name'],
    'path':folder['path'],
    'files':folder['filecount'],
    'volume(MB)':folder['volume']['MB'],
    },subfolders)) 
subfolderdetails_li=list(map(lambda folder: [
    folder['name'],
    folder['path'],
    folder['filecount'],
    folder['volume']['MB'],
    len(folder['subfolders'])
    ],subfolders)) 

In [30]:
subfolderdetails_li

[]

In [27]:
save_to_csv_filedetails(data=subfolderdetails_li,colums=["Names","Path","Files","Volume MB","subfolders"],csv_filename='reportcsv.csv',save_folder=today_collection_path)

Exception: list type data colums are required 